## 2. EDA

Goals:

- Select the variables for predictions.
- Discover other interesting trends.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 80)

In [4]:
%%time
training = pd.read_excel("../data/training_cleaned.xlsx")
test = pd.read_excel("../data/test_cleaned.xlsx")

Wall time: 26.4 s


### 2.1 Variable Selection

#### Variables that won't appear in actual prediction

- Debt settlement information. There won't be a debt settlment until a default occurs.
- Loan payment information. Our dataset only contains concluded loans. Loan payment on conluded loans contains information that wouldn't be available on on-going loans. For example, 

In [7]:
training.head()

,Loan Amount,Funded Amount,Funded Amount Investor,Term,Interest Rate,Installment,Grade,Sub Grade,Loan Status,Issued Date,Title,Description,Purpose,Employer Title,Employment Length,Home Ownership,Annual Income,Verification Status,Zip Code,State,Debt-to-income Ratio,Delinquencies in 2 years,Earliest Credit Line,Inquiries in 6 months,Months Since Last Delinquency,Months Since Last Public Record,Open Accounts,Derogatory Public Records,Revolving Balance,Revolving Balance Utilization,Number of Total Accounts,Last Credit Pulled Date,Public Bankcruptcy Records,Total Payment,Total Payment Investor,Total Received Principle,Total Received Interest,Total Received Late Fee,Recoveries,Collection Recovery Fee,Last Payment Date,Last Payment Amount,Debt Settlement Flag,Debt Settlement Flag Date,Settlement Status,Settlement Date,Settlement Amount,Settlement Percentage,Settlement Term,Credit History Length in Months,Statewide Unemployment at Issuance,GDP Quarterly Growth at Issuance,Industry
0,35000,35000,34975.0,60 months,0.1171,773.44,B,B3,1,2011-11-01,Restaurant Inventory,Loan proceeds will be used to partially fund a...,small_business,US Department of Labor,10+ years,MORTGAGE,110000.0,Verified,945xx,CA,1.06,0,1971-01-01,0,0,0,10,0,4142,0.064,27,2017-07-01,0,11601.60,11593.34,6926.82,4652.28,0.0,22.50,0.00,2013-02-01,773.44,N,NaT,NaN,NaT,NaN,NaN,NaN,490,11.3,4.7,Not Listed
1,9500,9500,9500.0,36 months,0.1465,327.70,C,C3,0,2011-11-01,familyneeds my help,i need this money to help my family in Thailan...,other,costco wholesales,10+ years,RENT,54000.0,Verified,334xx,FL,17.69,0,2001-05-01,1,0,0,6,0,5460,0.853,11,2018-10-01,0,9616.54,9616.54,9500.00,116.54,0.0,0.00,0.00,2011-12-01,9616.95,N,NaT,NaN,NaT,NaN,NaN,NaN,126,9.3,4.7,Not Listed
2,3800,3800,3800.0,36 months,0.0751,118.23,A,A3,1,2011-11-01,Motorcycle Loan,NaN,car,Five Guys,< 1 year,MORTGAGE,47000.0,Source Verified,132xx,NY,22.52,0,2002-06-01,3,0,0,10,0,8100,0.393,41,2012-08-01,0,1064.07,1064.07,869.42,191.95,0.0,2.70,0.00,2012-08-01,118.23,N,NaT,NaN,NaT,NaN,NaN,NaN,113,8.5,4.7,Not Listed
3,12400,12400,12400.0,60 months,0.2206,342.90,F,F4,1,2011-11-01,Debt Consolidation Loan,NaN,debt_consolidation,carmelo policaro construction,9 years,OWN,65004.0,Source Verified,077xx,NJ,6.26,0,2004-04-01,3,78,0,11,0,8990,0.775,21,2016-10-01,0,2127.63,2127.63,595.32,1116.48,0.0,415.83,4.43,2012-04-01,342.90,N,NaT,NaN,NaT,NaN,NaN,NaN,91,9.2,4.7,Not Listed
4,4000,4000,4000.0,60 months,0.1727,100.00,D,D3,1,2011-11-01,Medical,NaN,other,Tax Return Center,4 years,RENT,45000.0,Source Verified,331xx,FL,7.37,0,2003-02-01,0,50,0,10,0,4786,0.825,13,2016-10-01,0,829.14,829.14,309.36,388.82,0.0,130.96,1.39,2012-06-01,100.00,N,NaT,NaN,NaT,NaN,NaN,NaN,105,9.3,4.7,Not Listed


In [12]:
principleFullyPaid_bool = training["Total Received Principle"] == training["Funded Amount"]
training_principleFullyPaid = training[principleFullyPaid_bool]


In [23]:
variables_not_used = ["Debt Settlement Flag", "Debt Settlement Flag Date", "Settlement Status", "Settlement Date", "Settlement Amount", 
                     "Settlement Term", "Settlement Percentage",
                     "Total Payment", "Total Payment Investor", "Total Received Principle", "Total Received Interest", 
                      "Total Received Late Fee", "Recoveries", "Collection Recovery Fee", "Last Payment Date", "Last Payment Amount"
                     "Funded Amount", "Funded Amount Investor", 
                      "Grade", "Sub Grade",
                     "Industry"]
training_for_prediction = training.drop(columns = variables_not_used)
training_for_prediction.head(3)

,Loan Amount,Term,Interest Rate,Installment,Loan Status,Issued Date,Title,Description,Purpose,Employer Title,Employment Length,Home Ownership,Annual Income,Verification Status,Zip Code,State,Debt-to-income Ratio,Delinquencies in 2 years,Earliest Credit Line,Inquiries in 6 months,Months Since Last Delinquency,Months Since Last Public Record,Open Accounts,Derogatory Public Records,Revolving Balance,Revolving Balance Utilization,Number of Total Accounts,Last Credit Pulled Date,Public Bankcruptcy Records,Last Payment Date,Last Payment Amount,Credit History Length in Months,Statewide Unemployment at Issuance,GDP Quarterly Growth at Issuance
0,35000,60 months,0.1171,773.44,1,2011-11-01,Restaurant Inventory,Loan proceeds will be used to partially fund a...,small_business,US Department of Labor,10+ years,MORTGAGE,110000.0,Verified,945xx,CA,1.06,0,1971-01-01,0,0,0,10,0,4142,0.064,27,2017-07-01,0,2013-02-01,773.44,490,11.3,4.7
1,9500,36 months,0.1465,327.70,0,2011-11-01,familyneeds my help,i need this money to help my family in Thailan...,other,costco wholesales,10+ years,RENT,54000.0,Verified,334xx,FL,17.69,0,2001-05-01,1,0,0,6,0,5460,0.853,11,2018-10-01,0,2011-12-01,9616.95,126,9.3,4.7
2,3800,36 months,0.0751,118.23,1,2011-11-01,Motorcycle Loan,NaN,car,Five Guys,< 1 year,MORTGAGE,47000.0,Source Verified,132xx,NY,22.52,0,2002-06-01,3,0,0,10,0,8100,0.393,41,2012-08-01,0,2012-08-01,118.23,113,8.5,4.7


What we have to include:
- `Loan Amount`
- `Interest Rate`
- `Homeownership`
- `Annual Income` & what to do with `Verification Status`
- `Debt-to-income Ratio`
- `GDP Quarterly Growth at Issuance`, `Statewide Unemployment at Issuance`

What we need to decide on:

- `Installment`, `Term`, `Issued Date`
- `Title`, `Description`, `Purpose`
- `Employer Title`, `Employment Length`
- `Zipcode`, `State`, `Statewide Unemployment at Issuance`, `GDP Quarterly Growth at Issuance`
- late payments: `Delinquencies in 2 years`, `Months Since Last Delinquency`, `Months Since Last Public Record`, `Inquiries in 6 months`, `Open Accounts`, `Derogatory Public Records`, `Public Bankcruptcy Records`
- `Revolving Balance Utilization`, `Revolving Balance`